from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor

# Resource can be required for some backends, e.g. Jaeger
# If resource wouldn't be set - traces wouldn't appears in Jaeger
resource = Resource(attributes={
    "service.name": "service"
})

trace.set_tracer_provider(TracerProvider(resource=resource))
tracer = trace.get_tracer(__name__)

otlp_exporter = OTLPSpanExporter(endpoint="http://localhost:4317", insecure=True)

span_processor = BatchSpanProcessor(otlp_exporter)

trace.get_tracer_provider().add_span_processor(span_processor)

with tracer.start_as_current_span("foo"):
    print("Hello world!")

# Working with Spans

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
repo = Path().cwd().parent.parent.resolve()
sys.path.append(str(repo))

In [ ]:
from pprint import pformat
from pprint import pprint

from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp
from examples.expositional.end2end_apps.custom_app.custom_retriever import CustomRetriever
import pandas as pd

from trulens_eval import instruments
from trulens_eval.trace.category import Categorizer
from trulens_eval.tru_custom_app import TruCustomApp

In [ ]:
from trulens_eval import Tru
Tru().reset_database()
Tru().start_dashboard(_dev=repo, force=True)

In [ ]:
# Create custom app:
custom_app = CustomApp(delay=0.0, alloc=0)

# Create trulens wrapper:
tru_custom_app = TruCustomApp(
    custom_app,
    app_id="customapp",
)

custom_agent = custom_app.agents[0]

tru_custom_agent = TruCustomApp(
     custom_agent,
     app_id="customapp_agent"
)

In [ ]:
instruments.Instrument().print_instrumentation()

In [ ]:
tru_custom_app.print_instrumented()

In [ ]:
tru_custom_agent.print_instrumented()

In [ ]:
with tru_custom_app as recorder:
    custom_app.respond_to_query("testing")

rec = recorder.get()

In [ ]:
spans = Categorizer.spans_of_record(rec)

pd.DataFrame(
    [(
        s.trace_id & 0xff,
        s.name,
        type(s).__name__,
        s.span_type,
        s.span_id & 0xff,
        s.parent_span_id & 0xff if s.parent_span_id else -1,
        s.attributes
    ) for s in spans],
    columns=[
        "trace_id",
        "name",
        "type",
        "span_type",
        "span_id",
        "parent_span_id",
        "attributes"
    ],
)

In [ ]:
for span in spans:
    pprint(span)
    # pprint(span.model_dump())
    print()

In [ ]:
frozen = spans[1].freeze()

In [ ]:
frozen.links

In [ ]:
from trulens_eval.trace.span import Span

Span.thaw(frozen).model_dump()

In [ ]:
from examples.expositional.end2end_apps.custom_app.custom_tool import CustomStackTool
# setattr(CustomStackTool, "invoke", None)
getattr(CustomStackTool.invoke, "__tru_apps")

In [ ]:
dir(CustomStackTool.last_stack[0])

In [ ]:
import dis
for field in ["f_lasti", "f_lineno", "f_trace_lines", "f_trace_opcodes"]:#, "f_code", ]:
    print(CustomStackTool.last_stack[0].f_globals['__name__'])
    print(f"{field}: {getattr(CustomStackTool.last_stack[0], field)}")
    continue
    for cfield in [
        'co_argcount',
        'co_cellvars',
        'co_code',
        'co_consts',
        'co_exceptiontable',
        'co_filename',
        'co_firstlineno',
        'co_flags',
        'co_freevars',
        'co_kwonlyargcount',
        'co_lines',
        'co_linetable',
        'co_lnotab',
        'co_name',
        'co_names',
        'co_nlocals',
        'co_positions',
        'co_posonlyargcount',
        'co_qualname',
        'co_stacksize',
        'co_varnames'
    ]:
        if cfield == "co_code":
            print(f"  {cfield}: {dis.Bytecode(CustomStackTool.last_stack[0].f_code).dis()}")
        else:
            print(f"  {cfield}: {getattr(CustomStackTool.last_stack[0].f_code, cfield)}")

    

In [ ]:
b = dis.Bytecode(CustomStackTool.last_stack[0].f_code)

In [ ]:
import inspect
inspect.getmodule(CustomStackTool.last_stack[0].f_code).__name__

In [ ]:
for line in CustomStackTool.last_stack[0].f_code.co_lines():
    print(line)